In [6]:

import os
import re
def parse_filename_ex(filename):
    pattern = r"vl(\d+)_([0-9a-zA-Z]+)_stride_(\d+)_fs_(\d+)_([a-z]{2})((?:_[a-zA-Z]+_\d+)+)_hw_(\d+)_nf_(\d+)_(\w+)CPU__stats\.txt"
    match = re.match(pattern, filename)

    if match:
        num_bits = match.group(2)
        if num_bits == 'bin':
            num_bits = 1
        elif num_bits == '8bt':
            num_bits = 8
        else:
            raise Exception("number of bits not hardcoded")
            num_bits = int(num_bits)

        # Parsing and sorting auxiliary stationary and stats pairs
        aux_pairs_str = match.group(6)
        aux_pairs = re.findall(r"_([a-zA-Z]+)_(\d+)", aux_pairs_str)
        sorted_aux_pairs = sorted(aux_pairs, key=lambda x: x[0])

        aux_data = {}
        for i, (name, value) in enumerate(sorted_aux_pairs):
            if name not in aux_data:
                aux_data[f'aux_{name}'] = int(value)
            else:
                aux_data[f'aux_{name}_{i + 1}'] = int(value)

        return {
            'vector_length': int(match.group(1)),
            'num_bits': num_bits,
            'stride': int(match.group(3)),
            'filter_size': int(match.group(4)),
            'anchoring_stationary': match.group(5),
            'input_hw': int(match.group(7)),
            'num_filters': int(match.group(8)),
            'cpu_type': match.group(9),
            **aux_data
        }
    else:
        print(f"unmatched. filename: {filename}")
        return {}

directory = '/home/zhouzikai/nn_ext_dataflows/gen_programs/noblock/log'

 

In [7]:
for filename in os.listdir(directory):
    parse_res = parse_filename_ex(filename)
    print(parse_res)
    break

{'vector_length': 512, 'num_bits': 8, 'stride': 2, 'filter_size': 4, 'anchoring_stationary': 'ws', 'input_hw': 112, 'num_filters': 8, 'cpu_type': 'ArmO3', 'aux_is': 5, 'aux_os': 0}


In [16]:
total_count = 0
for filename in os.listdir(directory):
    parse_res = parse_filename_ex(filename)
    if "os_ws_" not in filename:
        continue
    if ((32 // (parse_res["vector_length"] // 128)) - 3) != (parse_res["aux_is"] +  parse_res["aux_ws"]):
        if parse_res["filter_size"]*parse_res["filter_size"] == parse_res["aux_ws"]:
            if parse_res["filter_size"]*(parse_res["filter_size"] -parse_res["stride"]) == parse_res["aux_is"]:
                continue
        total_count += 1
        os.remove(os.path.join(directory,filename))

print("deleted a total of "+str(total_count)+" files")

deleted a total of 208 files


In [14]:
og_directory = '/home/zhouzikai/nn_ext_dataflows/gen_programs/noblock/log_og'
difference = [item for item in os.listdir(og_directory) if item not in os.listdir(directory)]

print(difference)

['vl256_bin_stride_2_fs_5_os_ws_6_is_0_hw_56_nf_64_ArmO3CPU__stats.txt', 'vl256_bin_stride_2_fs_5_os_ws_6_is_0_hw_112_nf_16_ArmO3CPU__stats.txt', 'vl512_bin_stride_1_fs_4_os_ws_2_is_0_hw_56_nf_64_ArmO3CPU__stats.txt', 'vl256_bin_stride_1_fs_4_os_ws_7_is_0_hw_56_nf_8_ArmO3CPU__stats.txt', 'vl256_8bt_stride_1_fs_3_os_ws_9_is_3_hw_112_nf_16_ArmO3CPU__stats.txt', 'vl256_8bt_stride_1_fs_4_os_ws_7_is_0_hw_112_nf_16_ArmO3CPU__stats.txt', 'vl256_bin_stride_1_fs_4_os_ws_7_is_0_hw_112_nf_32_ArmO3CPU__stats.txt', 'vl512_8bt_stride_2_fs_4_os_ws_3_is_0_hw_56_nf_32_ArmO3CPU__stats.txt', 'vl128_bin_stride_1_fs_3_os_ws_9_is_6_hw_56_nf_16_ArmO3CPU__stats.txt', 'vl512_8bt_stride_2_fs_4_os_ws_3_is_0_hw_56_nf_16_ArmO3CPU__stats.txt', 'vl256_8bt_stride_2_fs_3_os_ws_9_is_1_hw_56_nf_16_ArmO3CPU__stats.txt', 'vl128_bin_stride_2_fs_5_os_ws_14_is_4_hw_112_nf_8_ArmO3CPU__stats.txt', 'vl256_bin_stride_1_fs_4_os_ws_7_is_0_hw_112_nf_16_ArmO3CPU__stats.txt', 'vl512_8bt_stride_1_fs_5_os_ws_0_is_0_hw_112_nf_16_ArmO3CP

In [15]:
total_count = 0
for filename in os.listdir(directory):
    parse_res = parse_filename_ex(filename)
    if "is_ws_" not in filename:
        continue
    if ((32 // (parse_res["vector_length"] // 128)) - 3) != (parse_res["aux_os"] +  parse_res["aux_ws"]):
        total_count += 1

print("deleted a total of "+str(total_count)+" files")

deleted a total of 96 files
